In [1]:
import pandas as pd

In [4]:
cleaned_data = pd.read_csv('C:/Users/dell/Documents/Guvi Lectures/Swiggy model/swiggy_cleaned_model.csv')

In [5]:
cleaned_data

,name,city,rating,cost,cuisine,address
0,AB FOODS POINT,Abohar,4.0,200.0,"Beverages,Pizzas","AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI..."
1,Janta Sweet House,Abohar,4.4,200.0,"Sweets,Bakery","Janta Sweet House, Bazar No.9, Circullar Road,..."
2,theka coffee desi,Abohar,3.8,100.0,Beverages,"theka coffee desi, sahtiya sadan road city"
3,Singh Hut,Abohar,3.7,250.0,"Fast Food,Indian","Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR"
4,GRILL MASTERS,Abohar,4.0,250.0,"Italian-American,Fast Food","GRILL MASTERS, ADA Heights, Abohar - Hanumanga..."
...,...,...,...,...,...,...
148393,The Food Delight,Yavatmal,4.0,200.0,"Fast Food,Snacks","The Food Delight, 94MC+X35, New Singhania Naga..."
148394,MAITRI FOODS & BEVERAGES,Yavatmal,4.0,300.0,Pizzas,"MAITRI FOODS & BEVERAGES, POLIC MITRYA SOCIETY..."
148395,Cafe Bella Ciao,Yavatmal,4.0,300.0,"Fast Food,Snacks","Cafe Bella Ciao, SHOP NO 2 NEMANI MARKET SBI S..."
148396,GRILL ZILLA,Yavatmal,4.0,250.0,Continental,"GRILL ZILLA, SHO NO 2/6, POSTEL GROUND CHOWPAT..."


In [6]:
cleaned_data = cleaned_data.drop(columns=["name","address"])

In [7]:
cleaned_data

,city,rating,cost,cuisine
0,Abohar,4.0,200.0,"Beverages,Pizzas"
1,Abohar,4.4,200.0,"Sweets,Bakery"
2,Abohar,3.8,100.0,Beverages
3,Abohar,3.7,250.0,"Fast Food,Indian"
4,Abohar,4.0,250.0,"Italian-American,Fast Food"
...,...,...,...,...
148393,Yavatmal,4.0,200.0,"Fast Food,Snacks"
148394,Yavatmal,4.0,300.0,Pizzas
148395,Yavatmal,4.0,300.0,"Fast Food,Snacks"
148396,Yavatmal,4.0,250.0,Continental


In [8]:
cleaned_data.isnull().sum()

city       0
rating     0
cost       0
cuisine    0
dtype: int64

In [9]:
cleaned_data.to_csv("swiggy_cleaned_revised.csv",index=False)
print("Done File Saved!")

Done File Saved!


In [10]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder

In [11]:
# Load the data
file_path = 'C:/Users/dell/Documents/Guvi Lectures/Swiggy model/swiggy_cleaned_revised.csv'
df = pd.read_csv(file_path)

# Display the first few rows
print('Initial DataFrame:')
print(df.head())

# Handling missing values
# Replace '--' with NaN and fill with the median for numerical columns
numerical_columns = ['rating', 'cost']
df.replace('--', np.nan, inplace=True)
df[numerical_columns] = df[numerical_columns].apply(pd.to_numeric, errors='coerce')
df.fillna(df.median(numeric_only=True), inplace=True)

# Apply One-Hot Encoding to categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_cuisines = encoder.fit_transform(df[['cuisine']])

# Save the encoder as a pickle file
with open('encoder_updated.pkl', 'wb') as file:
    pickle.dump(encoder, file)

# Convert the encoded data to a DataFrame
encoded_cuisines_df = pd.DataFrame(encoded_cuisines, columns=encoder.get_feature_names_out(['cuisine']))

# Drop original 'cuisine' column and concatenate the encoded DataFrame
processed_df = pd.concat([df.drop('cuisine', axis=1), encoded_cuisines_df], axis=1)

# Save the processed DataFrame as encoded_data.csv
processed_df.to_csv('encoded_data_updated.csv', index=False)

# Display the first few rows of the processed DataFrame
print('Processed DataFrame:')
print(processed_df.head())

print('✅ Data preprocessing completed successfully!')

Initial DataFrame:
     city  rating   cost                     cuisine
0  Abohar     4.0  200.0            Beverages,Pizzas
1  Abohar     4.4  200.0               Sweets,Bakery
2  Abohar     3.8  100.0                   Beverages
3  Abohar     3.7  250.0            Fast Food,Indian
4  Abohar     4.0  250.0  Italian-American,Fast Food


MemoryError: Unable to allocate 2.36 GiB for an array with shape (2132, 148398) and data type float64

In [13]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import save_npz, csr_matrix

# Load the cleaned data
file_path = 'C:/Users/dell/Documents/Guvi Lectures/Swiggy model//swiggy_cleaned_revised.csv'
df = pd.read_csv(file_path)

# Drop 'address' column as it's not useful for recommendation
if 'address' in df.columns:
    df = df.drop(columns=['address'])

# Handle missing values by filling with mode for categorical and median for numerical
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].median(), inplace=True)

# Apply One-Hot Encoding to categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
categorical_features = df[['city', 'cuisine']]
encoded_features = encoder.fit_transform(categorical_features)

# Save the encoder as a pickle file
with open('encoder_true_copy.pkl', 'wb') as file:
    pickle.dump(encoder, file)

# Combine encoded categorical features with numerical features
numerical_features = df.drop(columns=['city', 'cuisine']).astype('float32')
encoded_data = csr_matrix(np.hstack((numerical_features.values, encoded_features.toarray())))

# Save the sparse matrix
save_npz('encoded_data_sparse.npz', encoded_data)

print('✅ Sparse data generated and saved successfully!')

C:\Users\dell\AppData\Local\Temp\ipykernel_19276\3521837366.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
C:\Users\dell\AppData\Local\Temp\ipykernel_19276\3521837366.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

MemoryError: Unable to allocate 3.26 GiB for an array with shape (148398, 2953) and data type float64

In [14]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import save_npz, csr_matrix


# Load cleaned data
file_path = 'C:/Users/dell/Documents/Guvi Lectures/Swiggy model/swiggy_cleaned_revised.csv'
df = pd.read_csv(file_path)

# # Drop irrelevant columns
# if 'address' in df.columns:
#     df = df.drop(columns=['address'])

# # Convert 'cost' to numeric after cleaning currency symbols and commas
# if df['cost'].dtype == 'object':
#     df['cost'] = pd.to_numeric(df['cost'].replace('[₹,]', '', regex=True), errors='coerce')

# # Fill missing numeric values with the median
# if df['rating'].isnull().any():
#     df['rating'].fillna(df['rating'].median(), inplace=True)

# df.fillna(0, inplace=True)  # Fill any remaining NaN values with 0


# Initialize One-Hot Encoder
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True)


# Fit encoder on 'city' and 'cuisine'
categorical_features = df[['city', 'cuisine']]
encoded_features = encoder.fit_transform(categorical_features)


# Save the encoder as a pickle file
with open('encoder_today.pkl', 'wb') as file:
    pickle.dump(encoder, file)


# Save the sparse matrix
save_npz('encoded_data_sparse.npz', encoded_features)


print('✅ Sparse matrix created and saved successfully!')

✅ Sparse matrix created and saved successfully!
